In [101]:
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import yfinance as yf
import plotly.graph_objs as go
from finta import TA


In [102]:
# Create a function to pull in data using Yahoo finance API
def asset_df(*x):
    
    data = yf.download(tickers=[*x], period = '90d', interval = '1d')
    df = data['Close'].dropna()
    return df

In [103]:
df = asset_df('BTC-USD','ETH-USD','SOL1-USD','AVAX-USD','LUNA1-USD','BNB-USD','MATIC-USD')
df

[*********************100%***********************]  7 of 7 completed


,AVAX-USD,BNB-USD,BTC-USD,ETH-USD,LUNA1-USD,MATIC-USD,SOL1-USD
Date,,,,,,,
2021-09-14,51.578117,414.236237,47092.492188,3429.169678,35.666565,1.316790,158.094818
2021-09-15,57.625393,431.423035,48176.347656,3615.282715,36.689205,1.416373,159.117386
2021-09-16,67.842697,424.649231,47783.359375,3571.294922,37.522469,1.383580,152.474060
2021-09-17,69.934013,406.632812,47267.519531,3398.538818,34.425652,1.422436,147.593063
2021-09-18,69.012383,410.809296,48278.363281,3432.018311,34.781631,1.366242,169.184219
...,...,...,...,...,...,...,...
2021-12-08,93.221054,607.592773,50504.796875,4439.357910,75.037659,2.418730,194.621704
2021-12-09,85.461769,571.648987,47672.121094,4119.815918,66.476570,2.090278,181.178085
2021-12-10,80.739677,552.966858,47243.304688,3908.496094,60.930698,2.097674,167.954834


In [104]:
# Create function to run correlation matrix on any asset
def asset_corr(df):
    pct_chg_df = df.pct_change()
    df_corr = pct_chg_df.corr()
    df_corr_plot = df_corr.hvplot.heatmap()
    return df_corr_plot

In [105]:
asset_corr(df)

:HeatMap   [columns,index]   (value)

In [106]:
# Convert df to % change for analysis
pct_chg_df = df.pct_change()
pct_chg_df = pct_chg_df.dropna()

# RF Model

In [107]:
# Define features set
X = pct_chg_df.copy()
X.drop("MATIC-USD", axis=1, inplace=True)
X.head()

,AVAX-USD,BNB-USD,BTC-USD,ETH-USD,LUNA1-USD,SOL1-USD
Date,,,,,,
2021-09-15,0.117245,0.041490,0.023015,0.054273,0.028672,0.006468
2021-09-16,0.177306,-0.015701,-0.008157,-0.012167,0.022711,-0.041751
2021-09-17,0.030826,-0.042427,-0.010795,-0.048374,-0.082532,-0.032012
2021-09-18,-0.013179,0.010271,0.021386,0.009851,0.010341,0.146288
2021-09-19,0.015408,-0.005690,-0.021089,-0.029886,-0.052311,-0.098517


In [108]:
# Define target vector
y = pct_chg_df["MATIC-USD"].values.reshape(-1, 1)
y[:5]

array([[ 0.0756256 ],
       [-0.02315283],
       [ 0.02808369],
       [-0.03950543],
       [-0.03783814]])

In [109]:
# Create function for RF Model

def RF(X,y):
    
    #split data into training and testing data
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
    
    #initiate, fit, and train rf_model
    rf_model = RandomForestRegressor(n_estimators=500, random_state=42)
    rf_model = rf_model.fit(X_train, y_train)
    
    #make predictions for target using X_test
    predictions = rf_model.predict(X_test)
    
    return rf_model.score(X_test, y_test)

In [110]:
RF(X,y)

/Users/jonyamaoka/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.


0.6655135766075821

In [111]:
# # List the top 10 most important features
# importances_sorted = sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)
# importances_sorted[:10]

# Algo Trader

In [112]:
def bollBands(ticker):
    # Create a new clean copy of the signals_df DataFrame
    bb_signals_df = yf.download(tickers = ticker, period = '90d', interval = '1d')
    
    # Determine the Bollinger Bands for the Dataset
    bbands_df = TA.BBANDS(bb_signals_df)
    
    # Concatenate the Bollinger Bands to the DataFrame
    bb_signals_df = pd.concat([bb_signals_df, bbands_df], axis=1)
    
    # Visualize close price for the investment
    security_close = bb_signals_df[["Close"]].hvplot(
        line_color='lightgray',
        ylabel='Price in $',
        width=1000,
        height=400
    )

    bb_upper = bb_signals_df[["BB_UPPER"]].hvplot(
        line_color='purple',
        ylabel='Price in $',
        width=1000,
        height=400
    )


    bb_middle = bb_signals_df[["BB_MIDDLE"]].hvplot(
        line_color='orange',
        ylabel='Price in $',
        width=1000,
        height=400
    )

    bb_lower = bb_signals_df[["BB_LOWER"]].hvplot(
        line_color='blue',
        ylabel='Price in $',
        width=1000,
        height=400
    )

    # Overlay plots
    bbands_plot = security_close * bb_upper * bb_middle * bb_lower
    return bbands_plot

In [113]:
bollBands('ETH-USD')

[*********************100%***********************]  1 of 1 completed


:Overlay
   .Curve.I   :Curve   [Date]   (Close)
   .Curve.II  :Curve   [Date]   (BB_UPPER)
   .Curve.III :Curve   [Date]   (BB_MIDDLE)
   .Curve.IV  :Curve   [Date]   (BB_LOWER)

# Create Moving Averages

In [114]:
# Create a signals_df DataFrame that is a copy of the data Dataframe
signals_df = yf.download(tickers = 'BTC-USD', period = '250d', interval = '1d')

# Set the short window and long windows
short_window = 25
long_window = 100

# Add the SMA technical indicators for the short and long windows
signals_df["Short"] = TA.SMA(signals_df, short_window)
signals_df["Long"] = TA.SMA(signals_df, long_window)

# Determine the Bollinger Bands for the Dataset
bbands_df = TA.BBANDS(signals_df)
    
# Concatenate the Bollinger Bands to the DataFrame
all_signals_df = pd.concat([signals_df, bbands_df], axis=1)

# Review the DataFrame
all_signals_df.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Short,Long,BB_UPPER,BB_MIDDLE,BB_LOWER
Date,,,,,,,,,,,
2021-04-07,58186.507812,58731.144531,55604.023438,56048.937500,56048.937500,75645303584,NaN,NaN,NaN,NaN,NaN
2021-04-08,56099.914062,58338.738281,55879.085938,58323.953125,58323.953125,53053855641,NaN,NaN,NaN,NaN,NaN
2021-04-09,58326.562500,58937.046875,57807.863281,58245.003906,58245.003906,46655208546,NaN,NaN,NaN,NaN,NaN
2021-04-10,58253.777344,61276.664062,58038.707031,59793.234375,59793.234375,58238470525,NaN,NaN,NaN,NaN,NaN
2021-04-11,59846.230469,60790.554688,59289.796875,60204.964844,60204.964844,46280252580,NaN,NaN,NaN,NaN,NaN


In [115]:
# Set the Signal column
all_signals_df["Signal"] = 0.0

# Generate the trading signal 1 or 0,
# where 1 is when the Short window is greater than (or crosses over) the Long Window
# where 0 is when the Short window is under the Long window
all_signals_df["Signal"][short_window:] = np.where(
    all_signals_df["Short"][short_window:] > all_signals_df["Long"][short_window:], 1.0, 0.0)

# Calculate the points in time at which a position should be taken, 1 or -1
all_signals_df["Entry/Exit"] = all_signals_df["Signal"].diff()

# Review the DataFrame
all_signals_df.head()

/Users/jonyamaoka/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Open,High,Low,Close,Adj Close,Volume,Short,Long,BB_UPPER,BB_MIDDLE,BB_LOWER,Signal,Entry/Exit
Date,,,,,,,,,,,,,
2021-04-07,58186.507812,58731.144531,55604.023438,56048.937500,56048.937500,75645303584,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2021-04-08,56099.914062,58338.738281,55879.085938,58323.953125,58323.953125,53053855641,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2021-04-09,58326.562500,58937.046875,57807.863281,58245.003906,58245.003906,46655208546,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2021-04-10,58253.777344,61276.664062,58038.707031,59793.234375,59793.234375,58238470525,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2021-04-11,59846.230469,60790.554688,59289.796875,60204.964844,60204.964844,46280252580,NaN,NaN,NaN,NaN,NaN,0.0,0.0


In [117]:
# Visualize entry position relative to close price
entry = all_signals_df[all_signals_df["Entry/Exit"] == 1.0]["Close"].hvplot.scatter(
    color='purple',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize exit position relative to close price
exit = all_signals_df[all_signals_df["Entry/Exit"] == -1.0]["Close"].hvplot.scatter(
    color='orange',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = all_signals_df[["Close"]].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = all_signals_df[["Short", "Long"]].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = all_signals_df[["Close"]].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

bb_upper = all_signals_df[["BB_UPPER"]].hvplot(
    line_color='purple',
    ylabel='Price in $',
    width=1000,
    height=400
)


bb_middle = all_signals_df[["BB_MIDDLE"]].hvplot(
    line_color='orange',
    ylabel='Price in $',
    width=1000,
    height=400
)

bb_lower = all_signals_df[["BB_LOWER"]].hvplot(
    line_color='blue',
    ylabel='Price in $',
    width=1000,
    height=400
)
    
# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit * bb_upper * bb_middle * bb_lower
entry_exit_plot

:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)
   .Curve.II         :Curve   [Date]   (BB_UPPER)
   .Curve.III        :Curve   [Date]   (BB_MIDDLE)
   .Curve.IV         :Curve   [Date]   (BB_LOWER)

In [118]:
def algoTrader(ticker):
    
    # Create a signals_df DataFrame that is a copy of the data Dataframe
    signals_df = yf.download(tickers = ticker, period = '250d', interval = '1d')

    # Set the short window and long windows
    short_window = 25
    long_window = 100

    # Add the SMA technical indicators for the short and long windows
    signals_df["Short"] = TA.SMA(signals_df, short_window)
    signals_df["Long"] = TA.SMA(signals_df, long_window)

    # Determine the Bollinger Bands for the Dataset
    bbands_df = TA.BBANDS(signals_df)

    # Concatenate the Bollinger Bands to the DataFrame
    all_signals_df = pd.concat([signals_df, bbands_df], axis=1)

    # Set the Signal column
    all_signals_df["Signal"] = 0.0

    # Generate the trading signal 1 or 0,
    # where 1 is when the Short window is greater than (or crosses over) the Long Window
    # where 0 is when the Short window is under the Long window
    all_signals_df["Signal"][short_window:] = np.where(
        all_signals_df["Short"][short_window:] > all_signals_df["Long"][short_window:], 1.0, 0.0)

# Calculate the points in time at which a position should be taken, 1 or -1
all_signals_df["Entry/Exit"] = all_signals_df["Signal"].diff()

# Visualize entry position relative to close price
entry = all_signals_df[all_signals_df["Entry/Exit"] == 1.0]["Close"].hvplot.scatter(
    color='purple',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize exit position relative to close price
exit = all_signals_df[all_signals_df["Entry/Exit"] == -1.0]["Close"].hvplot.scatter(
    color='orange',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = all_signals_df[["Close"]].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = all_signals_df[["Short", "Long"]].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = all_signals_df[["Close"]].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

bb_upper = all_signals_df[["BB_UPPER"]].hvplot(
    line_color='purple',
    ylabel='Price in $',
    width=1000,
    height=400
)


bb_middle = all_signals_df[["BB_MIDDLE"]].hvplot(
    line_color='orange',
    ylabel='Price in $',
    width=1000,
    height=400
)

bb_lower = all_signals_df[["BB_LOWER"]].hvplot(
    line_color='blue',
    ylabel='Price in $',
    width=1000,
    height=400
)
    
# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit * bb_upper * bb_middle * bb_lower
entry_exit_plot

IndentationError: expected an indented block (<ipython-input-118-2b72a205fdc6>, line 4)